In [1]:
!pip install neurokit2 mne pandas numpy scikit-learn

  Using cached contourpy-1.3.1-cp312-cp312-macosx_11_0_arm64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.1/693.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 33.2 MB/s eta 0:00:00m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 29.9 MB/s eta 0:00:0031m31.7 MB/s eta 0:00:01
Using cached contourpy-1.3.1-cp312-cp312-macosx_11_0_arm64.whl (255 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import neurokit2 as nk
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import warnings

Matplotlib is building the font cache; this may take a moment.


In [3]:
warnings.filterwarnings("ignore")  # Suppress warnings for clean output

In [4]:
# Set Parameters
duration = 30  # Duration in seconds
fs = 1000  # Sampling frequency (Hz)

In [5]:
def generate_synthetic_data():
    ecg_signal = nk.ecg_simulate(duration=duration, sampling_rate=fs)
    eda_signal = nk.eda_simulate(duration=duration, sampling_rate=fs)
    eeg_alpha_signal = nk.eeg_simulate(duration=duration, sampling_rate=fs)
    eeg_beta_signal = nk.eeg_simulate(duration=duration, sampling_rate=fs)

    # Ensure signals have the same length
    min_length = min(len(ecg_signal), len(eda_signal), len(eeg_alpha_signal), len(eeg_beta_signal))
    return ecg_signal[:min_length], eda_signal[:min_length], eeg_alpha_signal[:min_length], eeg_beta_signal[:min_length]

In [6]:
# Process a single set of physiological signals
def process_signals(ecg_signal, eda_signal, eeg_alpha_signal, eeg_beta_signal):
    # Process ECG
    ecg_signals, ecg_info = nk.ecg_process(ecg_signal, sampling_rate=fs)
    
    # Process EDA
    eda_signals, eda_info = nk.eda_process(eda_signal, sampling_rate=fs)
    
    # EEG Power Analysis
    eeg_alpha_features = nk.eeg_power(eeg_alpha_signal, sampling_rate=fs)
    eeg_beta_features = nk.eeg_power(eeg_beta_signal, sampling_rate=fs)

    # Extract HRV (Heart Rate Variability) Features
    hrv_time_features = nk.hrv_time(ecg_info["ECG_R_Peaks"], sampling_rate=fs)
    hrv_freq_features = nk.hrv_frequency(ecg_info["ECG_R_Peaks"], sampling_rate=fs, psd_method="welch")

    # Calculate EEG Complexity (HFD - Higuchi Fractal Dimension)
    eeg_complexity_features = nk.complexity_hjorth(eeg_alpha_signal)
    hfd_alpha = eeg_complexity_features[0]

    return ecg_signals, eda_signals, eeg_alpha_features, eeg_beta_features, hrv_time_features, hrv_freq_features, hfd_alpha, eda_info

In [7]:
# Emotion Mapping Function
def classify_emotion(HRV_MeanNN, HRV_SDNN, HRV_RMSSD, HRV_pNN50, EDA_Phasic_Mean, EDA_Tonic_Mean, HRV_LF_HF, hfd_alpha):
    """
    Classifies emotions based on physiological signals using world-standard thresholds.
    """
    
    # 1️⃣ **High Stress / Anxiety (Sympathetic Dominance)**
    # - HRV Low: MeanNN < 750ms, SDNN < 30ms
    # - EDA Increased: Phasic > 0.3 µS, Tonic > 0.25 µS
    if HRV_MeanNN < 750 and HRV_SDNN < 30 or (EDA_Phasic_Mean > 0.3 and EDA_Tonic_Mean > 0.25):
        return "High Stress / Anxiety"
    
    # 2️⃣ **Fear / Nervousness (Fight or Flight Activation)**
    # - HRV Low: SDNN < 40ms, RMSSD < 25ms
    # - EDA: Tonic > 0.2 µS
    elif HRV_SDNN < 40 and HRV_RMSSD < 25 and EDA_Tonic_Mean > 0.2:
        return "Fear / Nervousness"
    
    # 3️⃣ **Emotional Distress (Physiological Dysregulation)**
    # - HRV: RMSSD < 35ms, pNN50 < 3%
    # - EDA: Phasic > 0.2 µS
    elif HRV_RMSSD < 35 and HRV_pNN50 < 3 and EDA_Phasic_Mean > 0.2:
        return "Emotional Distress"
    
    # 4️⃣ **Sympathetic Dominance (Chronic Stress Response)**
    # - HRV LF/HF Ratio > 2.5 (More sympathetic activation)
    # - EEG Complexity: HFD Alpha < 1.4 (Cognitive Impairment)
    elif HRV_LF_HF > 2.5 or hfd_alpha < 1.4:
        return "Sympathetic Dominance (Possible Anxiety)"
    
    # 5️⃣ **Neutral / Positive State**
    # - HRV Healthy: SDNN > 50ms, RMSSD > 40ms
    # - EDA Low: Phasic < 0.15 µS
    # - EEG Complexity: HFD Alpha > 2.0
    elif HRV_SDNN > 50 and HRV_RMSSD > 40 and EDA_Phasic_Mean < 0.15 and hfd_alpha > 2.0:
        return "Neutral / Positive State"
    
    # 6️⃣ **Calm & Rested (Parasympathetic Dominance)**
    # - HRV: SDNN > 60ms, RMSSD > 50ms
    # - EDA Low: Tonic < 0.1 µS
    elif HRV_SDNN > 60 and HRV_RMSSD > 50 and EDA_Tonic_Mean < 0.1:
            return "Calm & Rested (Parasympathetic Dominance)"
    
    else:
        return "Unclassified"

In [8]:
# Main Execution Block
if __name__ == "__main__":
    # Load real-world data or generate synthetic data
    use_real_data = False  # Set to True if using real biosignal data
    file_path = "real_biometric_data.csv"  # Change to actual file path

    if use_real_data:
        try:
            data = pd.read_csv(file_path)
            ecg_signal = data["ECG"].values
            eda_signal = data["EDA"].values
            eeg_alpha_signal = data["EEG_Alpha"].values
            eeg_beta_signal = data["EEG_Beta"].values
        except Exception as e:
            print(f"Error loading real-world data: {e}")
            exit()
    else:
        ecg_signal, eda_signal, eeg_alpha_signal, eeg_beta_signal = generate_synthetic_data()

    # Process the physiological signals
    ecg_signals, eda_signals, eeg_alpha_features, eeg_beta_features, hrv_time, hrv_freq, hfd_alpha, eda_info = process_signals(
        ecg_signal, eda_signal, eeg_alpha_signal, eeg_beta_signal
    )

    # Classify emotion
    # Extract necessary values safely
    HRV_MeanNN = hrv_time.get("HRV_MeanNN", pd.Series([np.nan])).values[0]
    HRV_SDNN = hrv_time.get("HRV_SDNN", pd.Series([np.nan])).values[0]
    HRV_RMSSD = hrv_time.get("HRV_RMSSD", pd.Series([np.nan])).values[0]
    HRV_pNN50 = hrv_time.get("HRV_pNN50", pd.Series([np.nan])).values[0]
    HRV_LF_HF = hrv_freq.get("HRV_LF", pd.Series([np.nan])).values[0] / max(hrv_freq.get("HRV_HF", pd.Series([1])).values[0], 1)
    
    EDA_Phasic_Mean = eda_info.get("EDA_Phasic_Mean", pd.Series([np.nan])).mean()
    EDA_Tonic_Mean = eda_info.get("EDA_Tonic_Mean", pd.Series([np.nan])).mean()
    
    detected_emotion = classify_emotion(HRV_MeanNN, HRV_SDNN, HRV_RMSSD, HRV_pNN50, EDA_Phasic_Mean, EDA_Tonic_Mean, HRV_LF_HF, hfd_alpha) 

    # Print Results
    print("\n💡 **Extracted Physiological Features**")
    print("HRV Time Features:\n", hrv_time)
    print("\nEDA Features:\n", eda_info)
    print("\nEEG Alpha Features:\n", eeg_alpha_features)
    print("\nEEG Beta Features:\n", eeg_beta_features)
    print("\nHRV Frequency Features:\n", hrv_freq)
    print("\nEEG Complexity Features (HFD):", hfd_alpha)
    print(f"\n🔥 **Detected Emotion: {detected_emotion}**")

    # Save processed data
    feature_data = pd.DataFrame({
        "HRV_MeanNN": [hrv_time.get("HRV_MeanNN", pd.Series([np.nan])).values[0]],
        "HRV_SDNN": [hrv_time.get("HRV_SDNN", pd.Series([np.nan])).values[0]],
        "HRV_RMSSD": [hrv_time.get("HRV_RMSSD", pd.Series([np.nan])).values[0]],
        "HRV_pNN50": [hrv_time.get("HRV_pNN50", pd.Series([np.nan])).values[0]],
        "EDA_Phasic_Mean": [eda_info.get("EDA_Phasic_Mean", pd.Series([np.nan])).mean()],
        "EDA_Tonic_Mean": [eda_info.get("EDA_Tonic_Mean", pd.Series([np.nan])).mean()],
        "HFD_Alpha": [hfd_alpha],
        "Emotion_Label": [detected_emotion]
    })

    feature_data.to_csv("processed_features.csv", index=False)
    print("\n✅ Processed physiological data saved as 'processed_features.csv'.")

Using default location ~/mne_data for sample...
Creating /Users/parags/mne_data


Failed to download 'MNE-sample-data-processed.tar.gz'. Will attempt the download again 2 more times.

  0%|                                              | 0.00/1.65G [00:00<?, ?B/s]IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
 31%|████████████▏                          | 515M/1.65G [11:44<25:54, 732kB/s]

%|                                      | 528k/1.65G [00:00<05:12, 5.28MB/s]
%|                                     | 1.06M/1.65G [00:00<09:24, 2.93MB/s]
%|                                     | 1.53M/1.65G [00:00<07:58, 3.45MB/s]
%|                                     | 1.99M/1.65G [00:00<07:14, 3.80MB/s]
%|                                     | 2.43M/1.65G [00:00<06:55, 3.98MB/s]
%|                                     | 2.86M/1.65G [00:00<07:15, 3.79MB/s]
%|                                     | 3.56M/1.65G [00:00<05:59, 4.59MB/s]
%|                                     | 4.48M/1.65G [00:00<04:39, 5.89MB/s]
%|                                     | 5.27M/

Attempting to create new mne-python configuration file:
/Users/parags/.mne/mne-python.json
Download complete in 13m49s (1576.2 MB)

💡 **Extracted Physiological Features**
HRV Time Features:
    HRV_MeanNN   HRV_SDNN  HRV_SDANN1  HRV_SDNNI1  HRV_SDANN2  HRV_SDNNI2  \
0   855.69697  11.663203         NaN         NaN         NaN         NaN   

   HRV_SDANN5  HRV_SDNNI5  HRV_RMSSD   HRV_SDSD  ...  HRV_IQRNN  HRV_SDRMSSD  \
0         NaN         NaN   12.33643  12.492578  ...       14.0     0.945428   

   HRV_Prc20NN  HRV_Prc80NN  HRV_pNN50  HRV_pNN20  HRV_MinNN  HRV_MaxNN  \
0        846.2        867.0        0.0   6.060606      825.0      874.0   

   HRV_HTI  HRV_TINN  
0      3.0     31.25  

[1 rows x 25 columns]

EDA Features:
 {'SCR_Onsets': array([108]), 'SCR_Peaks': array([468]), 'SCR_Height': array([0.8883525]), 'SCR_Amplitude': array([1.01878225]), 'SCR_RiseTime': array([0.36]), 'SCR_Recovery': array([989.]), 'SCR_RecoveryTime': array([0.521]), 'sampling_rate': 1000}

EEG Alpha